In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout,experimental
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import tensorflow as tf

import numpy as np
import pandas as pd
import os

import cv2
import matplotlib.pyplot as plt

In [2]:
train_path="train"
test_path="test"
val_path="validate"

In [3]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_train.append(img_arr)

x_test=[]
for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)
        
        img_arr=cv2.resize(img_arr,(224,224))

        x_val.append(img_arr)

In [4]:
# Generate Lable 
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:

training_set = train_datagen.flow_from_directory("train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 149 images belonging to 5 classes.
Found 69 images belonging to 5 classes.
Found 67 images belonging to 5 classes.


In [6]:
train_y = training_set.classes
test_y = test_set.classes
val_y = val_set.classes

In [7]:
train_y = training_set.classes
test_y = test_set.classes
val_y = val_set.classes

In [9]:
IMG_SIZE = 224
def augment(image, label):
    img = tf.cast(image, tf.float32)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = (img / 255.0)
    img = tf.image.random_crop(img, size=[IMG_SIZE, IMG_SIZE, 3])
    img = tf.image.random_brightness(img, max_delta=0.5)
    return img, label  

In [10]:
#Data Augmentation
augmented_images = list(map(augment,x_train,train_y))

In [11]:
for image in augmented_images:
    x_train.append(image[0])
    train_y = np.append(train_y,image[1])

In [12]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [13]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [14]:
lables = {}
lables[0] = "bridge"
lables[1] = "brokenPipe"
lables[2] = "fallenPole"
lables[3] = "garbage"
lables[4] = "pitHole"
lables

{0: 'bridge', 1: 'brokenPipe', 2: 'fallenPole', 3: 'garbage', 4: 'pitHole'}

In [27]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(input_shape=(224,224,3),include_top=False)
# base_model.trainable = True

In [28]:
for layer in base_model.layers:
    layer.trainable = False

In [29]:
x = Flatten()(base_model.output)

prediction = Dense(len(training_set.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

In [16]:
# from tensorflow.keras import layers

# inputs = layers.Input(shape = (224,224,3), name='inputLayer')

# x = base_model(inputs, training = False)
# x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
# x = layers.Dense(5, name='outputLayer')(x)
# outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x)

# model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")


In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 224, 224, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_2[0][0]            
______________________________________________________________________________________________

block5b_expand_bn (BatchNormali (None, 14, 14, 672)  2688        block5b_expand_conv[0][0]        
__________________________________________________________________________________________________
block5b_expand_activation (Acti (None, 14, 14, 672)  0           block5b_expand_bn[0][0]          
__________________________________________________________________________________________________
block5b_dwconv (DepthwiseConv2D (None, 14, 14, 672)  16800       block5b_expand_activation[0][0]  
__________________________________________________________________________________________________
block5b_bn (BatchNormalization) (None, 14, 14, 672)  2688        block5b_dwconv[0][0]             
__________________________________________________________________________________________________
block5b_activation (Activation) (None, 14, 14, 672)  0           block5b_bn[0][0]                 
__________________________________________________________________________________________________
block5b_se

In [18]:

for layer in model.layers[1].layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

In [19]:
for lnum, layer in enumerate(model.layers):
    print(lnum, layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

0 inputLayer True float32 <Policy "float32">
1 efficientnetb0 True float32 <Policy "float32">
2 poolingLayer True float32 <Policy "float32">
3 outputLayer True float32 <Policy "float32">
4 activationLayer True float32 <Policy "float32">


In [33]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [34]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)


Epoch 1/10
10/10 [==============================] - 31s 1s/step - loss: 9.8105 - accuracy: 0.2248 - val_loss: 16.3944 - val_accuracy: 0.2090
Epoch 2/10
10/10 [==============================] - 11s 1s/step - loss: 9.4290 - accuracy: 0.2181 - val_loss: 6.2888 - val_accuracy: 0.2239
Epoch 3/10
10/10 [==============================] - 11s 1s/step - loss: 5.0477 - accuracy: 0.2517 - val_loss: 4.0750 - val_accuracy: 0.2239
Epoch 4/10
10/10 [==============================] - 11s 1s/step - loss: 3.8990 - accuracy: 0.2081 - val_loss: 3.4926 - val_accuracy: 0.1940
Epoch 5/10
10/10 [==============================] - 11s 1s/step - loss: 3.1301 - accuracy: 0.2215 - val_loss: 2.5348 - val_accuracy: 0.2239
Epoch 6/10
10/10 [==============================] - 11s 1s/step - loss: 2.1319 - accuracy: 0.1946 - val_loss: 1.7255 - val_accuracy: 0.1940
Epoch 7/10
10/10 [==============================] - 11s 1s/step - loss: 1.8965 - accuracy: 0.2181 - val_loss: 2.1556 - val_accuracy: 0.1940
Epoch 8/10
10/10 [=